# Hello World

This notebook walks through basic code examples for integrating various packages with Neo4j, including `py2neo`, `ipython-cypher`, `pandas`, `networkx`, `igraph`, and `jgraph`.

# py2neo

`py2neo` is one of Neo4j's Python drivers. It offers a fully-featured interface for interacting with your data in Neo4j. Install `py2neo` with `pip install py2neo`.

## Connect

Connect to Neo4j with the `Graph` class.

In [1]:
from py2neo import Graph

graph = Graph("http://localhost:7474", user='EvaArevalo', password='yhwh123')

In [2]:
graph.delete_all()

## Nodes

Create nodes with the `Node` class. The first argument is the node's label. The remaining arguments are an arbitrary amount of node properties or key-value pairs.

In [3]:
from py2neo import Node

nicole = Node("Person", name="Nicole", age=24)
drew = Node("Person", name="Drew", age=20)

mtdew = Node("Drink", name="Mountain Dew", calories=9000)
cokezero = Node("Drink", name="Coke Zero", calories=0)

coke = Node("Manufacturer", name="Coca Cola")
pepsi = Node("Manufacturer", name="Pepsi")

graph.create(nicole | drew | mtdew | cokezero | coke | pepsi)

In [4]:
coke['name']

'Coca Cola'

In [5]:
type(cokezero.labels)


py2neo.cypher.encoding.LabelSetView

In [6]:
import scripts.vis
from scripts.vis import draw

options = {"Person": "name", "Drink": "name", "Manufacturer": "name"}
draw(graph, options)

P.S. - If you want to check out what's going on behind the scenes for the `draw()` function used above, take a look at [`scripts/vis.py`](https://github.com/nicolewhite/neo4j-jupyter/blob/master/scripts/vis.py).

## Relationships

Create relationships between nodes with the `Relationship` class.

In [7]:
from py2neo import Relationship

rel1=graph.create(Relationship(nicole, "LIKES", cokezero))
graph.create(Relationship(nicole, "LIKES", mtdew))
graph.create(Relationship(drew, "LIKES", mtdew))
graph.create(Relationship(coke, "MAKES", cokezero))
graph.create(Relationship(pepsi, "MAKES", mtdew))

draw(graph, options)

In [9]:
rel1=Relationship(coke, "MAKES", cokezero)

In [10]:
rel1

(Coca Cola)-[:MAKES {}]->(Coke Zero)

In [11]:
str1=str(rel1.relationships).split(':')
str1=str1[1].split('{')
str1=str1[0].strip(' ')
str1

'MAKES'

In [86]:
rel1.relationships.__getattr__

AttributeError: 'tuple' object has no attribute '__getattr__'

In [80]:
dir(rel1.relationships)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index']

In [66]:
from py2neo import data

dir(rel1)

['_Subgraph__nodes',
 '_Subgraph__relationships',
 '_Walkable__sequence',
 '__add__',
 '__and__',
 '__bool__',
 '__class__',
 '__contains__',
 '__db_create__',
 '__db_delete__',
 '__db_exists__',
 '__db_merge__',
 '__db_pull__',
 '__db_push__',
 '__db_separate__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__uuid__',
 '__walk__',
 '__weakref__',
 '__xor__',
 '_stale',
 'cast',
 'clear',
 'copy',
 'end_node',
 'fromkeys',
 'get',
 'graph',
 'identity',
 'items',
 'keys',
 'labels',
 'nodes',
 'pop',
 'popitem',
 'relationships',
 'setdefault',
 'start_node',
 'type',
 't

In [27]:
query = """
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name AS name, drink.name AS drink
"""

data = graph.run(query)

for d in data:
    print(d)

<Record name='Drew' drink='Mountain Dew'>
<Record name='Nicole' drink='Mountain Dew'>
<Record name='Nicole' drink='Coke Zero'>


## Parameterized Cypher

Pass parameters to Cypher queries by passing additional key-value arguments to `Graph.cypher.execute.` Parameters in Cypher are named and are wrapped in curly braces.

In [7]:
query = """
MATCH (p:Person)-[:LIKES]->(drink:Drink)
WHERE p.name = {name}
RETURN p.name AS name, AVG(drink.calories) AS avg_calories
"""

data = graph.run(query, name="Nicole")

for d in data:
    print(d)

<Record name='Nicole' avg_calories=4500.0>


# ipython-cypher

`ipython-cypher` exposes `%cypher` magic in Jupyter. Install `ipython-cypher` with `pip install ipython-cypher`.

In [8]:
%load_ext cypher

## Cypher

`%cypher` is intended for one-line Cypher queries and `%%cypher` is intended for multi-line Cypher queries. Placing `%%cypher` above a Cypher query will display that query's results.

In [9]:
%%cypher
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name, drink.name, drink.calories

Format: (http|https)://username:password@hostname:port/db/name


StatusException: Code [401]: Unauthorized. No permission -- see authorization schemes.
Authorization Required

## Pandas Data Frames

Cypher query results can be coerced to `pandas` data frames with the `get_dataframe` method. To assign Cypher query results to a variable, you need to use `%cypher` and separate lines with \\. You'll first need to install `pandas` with `pip install pandas`.

In [10]:
results = %cypher MATCH (person:Person)-[:LIKES]->(drink:Drink) \
                  RETURN person.name AS name, drink.name AS drink
    
df = results.get_dataframe()

df

Format: (http|https)://username:password@hostname:port/db/name


StatusException: Code [401]: Unauthorized. No permission -- see authorization schemes.
Authorization Required

In [11]:
df.index

NameError: name 'df' is not defined

In [12]:
df.iloc[[1]]

NameError: name 'df' is not defined

In [13]:
df["name"]

NameError: name 'df' is not defined

## NetworkX Graphs

Cypher query results can be coerced to `NetworkX` MultiDiGraphs, graphs that permit multiple edges between nodes, with the `get_graph` method. You'll first need to install `NetworkX` with `pip install networkx`.

In [14]:
import networkx as nx
%matplotlib inline

results = %cypher MATCH p = (:Person)-[:LIKES]->(:Drink) RETURN p

g = results.get_graph()

nx.draw(g)

Format: (http|https)://username:password@hostname:port/db/name


StatusException: Code [401]: Unauthorized. No permission -- see authorization schemes.
Authorization Required

In [15]:
g.nodes(data=True)

NameError: name 'g' is not defined

In [16]:
nx.degree(g)

NameError: name 'g' is not defined

# igraph

Cypher query results can be imported into `igraph` with `py2neo`. You'll need to install `igraph` with `pip install python-igraph`. Query results should be returned as edgelists, as `igraph` has a method for building an `igraph` object from a list of tuples representing edges between nodes.

In [17]:
from py2neo import Graph as PGraph
from igraph import Graph as IGraph

neo4j = PGraph()

query = """
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name AS source, drink.name AS target
"""

data = neo4j.run(query)
tups = []

for d in data:
    tups.append((d["source"], d["target"]))

DeprecationWarning: To avoid name collision with the igraph project, this visualization library has been renamed to 'jgraph'. Please upgrade when convenient.

In [18]:
ig = IGraph.TupleList(tups)

ig

NameError: name 'IGraph' is not defined

In [19]:
best = ig.vs.select(_degree = ig.maxdegree())["name"]
best

NameError: name 'ig' is not defined

# jgraph

`jgraph` will plot tuple lists as 3D graphs.

In [20]:
import jgraph

jgraph.draw([(1, 2), (2, 3), (3, 4), (4, 1), (4, 5), (5, 2)])

In [21]:
data = graph.run("MATCH (n)-->(m) RETURN ID(n), ID(m)")
data = [tuple(x) for x in data]

jgraph.draw(data)